# 손실함수

> 2.2.5 장에 해당하는 코드

## 출력값을 확률로 표현하기

In [1]:
# 코드 2-7

import torch
import torch.nn as nn

torch.manual_seed(70)

class Network(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()
        # 층을 구성
        # input layer > hidden layer 
        self.linear_ih = nn.Linear(in_features=input_size, 
                                   out_features=hidden_size)
        # hidden layer > output layer
        self.linear_ho = nn.Linear(in_features=hidden_size, 
                                   out_features=output_size)
        # activation layer
        self.activation_layer = nn.Sigmoid()
        
    def forward(self, x):
        z1 = self.linear_ih(x)
        a1 = self.activation_layer(z1)
        z2 = self.linear_ho(a1)
        y = self.activation_layer(z2)
        return y

# 입력텐서 생성    
x = torch.Tensor([[0, 1]])

# 커스텀 모듈 호출
net = Network(input_size=2, hidden_size=2, output_size=1)
y = net(x)
print(y.item())

0.42035338282585144


## 확률론적 접근

### 엔트로피

In [2]:
# 코드 2-8

P = torch.Tensor([0.4, 0.6])
Q = torch.Tensor([0.0, 1.0])

def self_information(x):
    return -torch.log(x)

def entropy(x):
    # log(0) = NaN 값이 나옴으로 아주 작은 수를 더해서 이를 방지한다.
    e = 1e-30
    return torch.sum((x+e)*self_information(x+e))


# 앞면이 40%, 뒷면이 60% 확률의 동전
print(entropy(P).numpy().round(4))
# 뒷면만 100% 나오는 확실한 동전
print(entropy(Q).numpy().round(4))

0.673
0.0


### KL-divergence

In [3]:
# 코드 2-9

def KL_divergence(q, p):
    """
    q: predict prob
    p: target prob
    """
    # log(0) = NaN 값이 나옴으로 아주 작은 수(e)를 더해서 이를 방지한다.
    e = 1e-30
    return torch.sum((p+e)*torch.log(p+e) - (p+e)*torch.log(q+e))

U = torch.Tensor([0.5, 0.5])
# 확률분포 P 와 U 의 KL-divergence
print(KL_divergence(P, U))
# 확률분포 Q 와 U 의 KL-divergence
print(KL_divergence(Q, U))

tensor(0.0204)
tensor(33.8456)


In [4]:
# 코드 2-10

loss_function = nn.KLDivLoss(reduction="sum")
e = 1e-30
## pytorch: y*(log(y) - x) = log(y)*y - x*y
# 확률분포 P 와 U 의 KL-divergence
print(loss_function(torch.log(P+e), U+e))
# 확률분포 Q 와 U 의 KL-divergence
print(loss_function(torch.log(Q+e), U+e))

tensor(0.0204)
tensor(33.8456)


In [5]:
# 코드 2-11

torch.manual_seed(70)

# 입력과 타겟텐서 생성
x = torch.Tensor([[0, 1]])
t = torch.Tensor([1])
# 이전에 만든 XOR 네트워크 호출
net = Network(input_size=2, hidden_size=2, output_size=1)
y = net(x)
# 타겟값의 원-핫 인코딩 생성
one_hot = torch.eye(2)
prob_t = one_hot.index_select(dim=0, index=t.long())
# 예측값도 확률 분포로 만들어준다. y=1이 될 확률분포이다.
prob_y = torch.cat([1-y, y], dim=1)
# t=1 에 해당하는 t의 확률분포와 예측값 y의 확률분포
print(prob_t)
print(prob_y)
# KL-divergence 구하기
loss_function = nn.KLDivLoss(reduction="sum")
print(loss_function(torch.log(prob_y), prob_t))

tensor([[0., 1.]])
tensor([[0.5796, 0.4204]], grad_fn=<CatBackward>)
tensor(0.8667, grad_fn=<KlDivBackward>)


### BCE Loss

In [6]:
# 코드 2-12

torch.manual_seed(70)

# 입력과 타겟텐서 생성
x = torch.Tensor([[0, 1]])
t = torch.Tensor([1])
# 이전에 만든 XOR 네트워크 호출
net = Network(input_size=2, hidden_size=2, output_size=1)
y = net(x)
# BCE 구하기
loss_function = nn.BCELoss(reduction="sum")
# 예측 값: y=1 확률 / 타겟 값: t=1 의 확률은 1
print(loss_function(y.squeeze(1), t))

tensor(0.8667, grad_fn=<BinaryCrossEntropyBackward>)


### Softmax

In [7]:
# 코드 2-13

torch.manual_seed(70)
# 선형결합값: 임의의 크기가 (1,10)인 벡터 텐서 생성
z = torch.rand(1, 10)
# Softmax 
y = torch.softmax(z, dim=1)
print(y)

tensor([[0.0986, 0.0914, 0.0897, 0.0831, 0.1241, 0.0729, 0.1315, 0.1243, 0.0711,
         0.1133]])


### Cross Entropy Loss

In [8]:
# 코드 2-14

loss_function = nn.CrossEntropyLoss(reduction="sum")
# 타겟값 또한 torch.LongTensor로 만들어야 한다.
print(loss_function(z, t.long()))

tensor(2.3930)
